In [40]:
import numpy as np
import praw
import datetime
from nltk import tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# bokeh
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool, PanTool
from bokeh.plotting import figure, show

from CryptoSentWebApp import config

In [19]:
conf = config.get_secret('tst/CryptoSentWebApp/Reddit')

In [20]:
output_notebook()

Loading BokehJS ...

In [21]:
sid = SentimentIntensityAnalyzer()

In [22]:
reddit = praw.Reddit(
    client_id = conf['client_id'],
    client_secret = conf['client_secret'],
    username = conf['username'],
    password = conf['password'],
    user_agent = conf['user_agent']
)

In [23]:
subreddit = reddit.subreddit('cryptocurrency')

In [51]:
for comment in subreddit.stream.comments():
    comment_id = comment.id
    comment_author = comment.author
    comment_link = comment.link_id
    comment_body = comment.body
    comment_ts = datetime.datetime.utcfromtimestamp(comment.created_utc)
    print(f'Id:{comment_id} Author:{comment_author} Submission:{comment_link}\nComment: {comment_body}\n')
    comment_sentences = tokenize.sent_tokenize(comment_body)
    for sentence in comment_sentences:    
        senti = sid.polarity_scores(sentence)
        print(f'{comment_ts}: {sentence} [{senti}]')
    print('\n')

Id:fl3a1vb Author:DramaticFirefighter8 Submission:t3_flwg79
Comment: I would learn the language of the country I live in if I was you. Besides, Italian is beautiful language!

2020-03-21 07:09:04: I would learn the language of the country I live in if I was you. [{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
2020-03-21 07:09:04: Besides, Italian is beautiful language! [{'neg': 0.0, 'neu': 0.488, 'pos': 0.512, 'compound': 0.636}]


Id:fl3a8ee Author:BrugelNauszmazcer Submission:t3_fmaqal
Comment: Batcoif

2020-03-21 07:12:40: Batcoif [{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]


Id:fl3a9am Author:vegasluna Submission:t3_fm5qsb
Comment: good point.

2020-03-21 07:13:10: good point. [{'neg': 0.0, 'neu': 0.256, 'pos': 0.744, 'compound': 0.4404}]


Id:fl3b0sa Author:PoliticsRealityTV Submission:t3_fm046p
Comment: no, "dumping" simply means they sold off their positions

2020-03-21 07:28:35: no, "dumping" simply means they sold off their positions [{'neg': 0.216, 'neu': 

Id:fl3mc91 Author:iamspencer1337 Submission:t3_fls7c3
Comment: Hive was started by the community. There is no central authority to shutdown your posts.

Of course they can add code to their frontend website that won't display your posts ( \*for instance, if you attempt to post CP\* ), but in terms of dissenting opinions, gathering from the reason they did this fork in the first place, I highly doubt they will censor anything that isn't outright illegal. You may get downvoted, but that's just one freedom-of-speech working with another ;\^)

2020-03-21 11:36:26: Hive was started by the community. [{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
2020-03-21 11:36:26: There is no central authority to shutdown your posts. [{'neg': 0.21, 'neu': 0.667, 'pos': 0.124, 'compound': -0.2263}]
2020-03-21 11:36:26: Of course they can add code to their frontend website that won't display your posts ( \*for instance, if you attempt to post CP\* ), but in terms of dissenting opinions, gathering f

KeyboardInterrupt: 

In [8]:
def moving_average(window, t=10, start=0):
    if len(window) < t: 
        return [start]
    return np.convolve(window[-t:], np.ones(t, dtype=float), mode="valid") / t

In [124]:
def subreddit_sentiment(subreddit):
    ma_data = ColumnDataSource(dict(timestamp=[], sentiment_ma=[]))    
    sentence_data = ColumnDataSource(dict(timestamp=[], sentiment=[]))
    hover = HoverTool(
        tooltips=[('time', '@timestamp{%Y-%m-%d %H:%M:%S}'), ('sentiment', '@sentiment_ma{%0.3f}')],
        formatters={'timestamp': 'datetime', 'sentiment_ma': 'printf'},
        mode='vline'
    )
    p = figure(
        plot_width=800,
        plot_height=400,
        x_axis_type='datetime',
        tools=[hover, PanTool(dimensions='height')],
        title="Real-Time sentiment plot",
        y_range=(-1, 1)
    )
    dots = p.circle(source=sentence_data, x='timestamp', y='sentiment')
    lin = p.line(source=ma_data, x='timestamp', y='sentiment_ma', line_color='red', line_width=2)
    p.xaxis.axis_label = "Time"
    p.yaxis.axis_label = "Reddit comment sentiment score"

    target = show(p, notebook_handle=True)
    subreddit = reddit.subreddit(subreddit)
    total_sentiment = []
    for comment in subreddit.stream.comments():
        comment_sent = tokenize.sent_tokenize(comment.body)
        sentence_sentiment = []
        for sentence in comment_sent:
            # Extract the sentiment from the sentence
            sentence_sentiment.append(sid.polarity_scores(sentence)['compound'])
            
        avg_sentence_sentiment = sum(sentence_sentiment) / len(sentence_sentiment)
        total_sentiment.append(avg_sentence_sentiment)
        sentence_data.stream(dict(
            timestamp=[datetime.datetime.utcfromtimestamp(comment.created_utc)],
            sentiment=[avg_sentence_sentiment]
        ), 1000)
        push_notebook(handle=target)    
        
        if len(total_sentiment) > 100:
            ma_data.stream(dict(
                timestamp=[datetime.datetime.utcfromtimestamp(comment.created_utc)],
                sentiment_ma=[moving_average(sentence_data.data['sentiment'], 100)]
            ), 11)
            total_sentiment=[]
            push_notebook(handle=target)

In [125]:
subreddit_sentiment('all')

KeyboardInterrupt: 

In [11]:
sid = SentimentIntensityAnalyzer()
for submission in subreddit.stream.submissions():
    sent = sid.polarity_scores(submission.title)
    print(submission.title, sent)

NameError: name 'subreddit' is not defined